# Summed Likelihood Analysis with Python

This sample analysis shows a way of performing joint likelihood on two data selections using the same XML model. This is useful if you want to do the following:

* Coanalysis of Front and Back selections (not using the combined IRF)
* Coanalysis of separate time intervals
* Coanalysis of separate energy ranges
* Pass 8 PSF type analysis
* Pass 8 EDISP type analysis

This tutorial also assumes that you've gone through the standard [binned likelihood](https://fermi.gsfc.nasa.gov/ssc/data/analysis/scitools/binned_likelihood_tutorial.html) thread using the combined front + back events, to which we will compare.

# Get the data

For this thread the original data were extracted from the [LAT data server](https://fermi.gsfc.nasa.gov/cgi-bin/ssc/LAT/LATDataQuery.cgi) with the following selections (these selections are similar to those in the paper):

```
Search Center (RA,Dec) = (193.98,-5.82)
Radius = 15 degrees
Start Time (MET) = 239557417 seconds (2008-08-04T15:43:37)
Stop Time (MET) = 302572802 seconds (2010-08-04T00:00:00)
Minimum Energy = 100 MeV
Maximum Energy = 500000 MeV
```

For more information on how to download LAT data please see the [Extract LAT Data](https://fermi.gsfc.nasa.gov/ssc/data/analysis/scitools/extract_latdata.html) tutorial.

These are the event files. Run the code cell below to retrieve them:
```
L181126210218F4F0ED2738_PH00.fits (5.4 MB)
L181126210218F4F0ED2738_PH01.fits (10.8 MB)
L181126210218F4F0ED2738_PH02.fits (6.9 MB)
L181126210218F4F0ED2738_PH03.fits (9.8 MB)
L181126210218F4F0ED2738_PH04.fits (7.8 MB)
L181126210218F4F0ED2738_PH05.fits (6.6 MB)
L181126210218F4F0ED2738_PH06.fits (4.8 MB)
L181126210218F4F0ED2738_SC00.fits (256 MB spacecraft file)
```

In [ ]:
!wget https://fermi.gsfc.nasa.gov/ssc/data/analysis/scitools/data/BinnedLikelihood/L181126210218F4F0ED2738_PH00.fits
!wget https://fermi.gsfc.nasa.gov/ssc/data/analysis/scitools/data/BinnedLikelihood/L181126210218F4F0ED2738_PH01.fits
!wget https://fermi.gsfc.nasa.gov/ssc/data/analysis/scitools/data/BinnedLikelihood/L181126210218F4F0ED2738_PH02.fits
!wget https://fermi.gsfc.nasa.gov/ssc/data/analysis/scitools/data/BinnedLikelihood/L181126210218F4F0ED2738_PH03.fits
!wget https://fermi.gsfc.nasa.gov/ssc/data/analysis/scitools/data/BinnedLikelihood/L181126210218F4F0ED2738_PH04.fits
!wget https://fermi.gsfc.nasa.gov/ssc/data/analysis/scitools/data/BinnedLikelihood/L181126210218F4F0ED2738_PH05.fits
!wget https://fermi.gsfc.nasa.gov/ssc/data/analysis/scitools/data/BinnedLikelihood/L181126210218F4F0ED2738_PH06.fits
!wget https://fermi.gsfc.nasa.gov/ssc/data/analysis/scitools/data/BinnedLikelihood/L181126210218F4F0ED2738_SC00.fits

In [ ]:
!mkdir data
!mv *.fits ./data

You'll first need to make a file list with the names of your input event files:

In [ ]:
!ls ./data/*_PH*.fits > ./data/binned_events.txt
!cat ./data/binned_events.txt

In the following analysis we've assumed that you've named your list of data files `binned_events.txt`.

# Perform Event Selections

You could follow the unbinned likelihood tutorial to perform your event selections using **gtlike*, **gtmktime**, etc. directly from the command line, and then use pylikelihood later.

But we're going to go ahead and use python. The `gt_apps` module provides methods to call these tools from within python. This'll get us used to using python.

So, let's jump into python:

In [ ]:
import gt_apps as my_apps

We need to run **gtselect** (called `filter` in python) twice. Once, we select only the front events and the other time we select only back events. You do this with `evtype=1` (front) and `evtype=2` (back).

In [ ]:
my_apps.filter['evclass'] = 128
my_apps.filter['evtype'] = 1
my_apps.filter['ra'] = 193.98
my_apps.filter['dec'] = -5.82
my_apps.filter['rad'] = 15
my_apps.filter['emin'] = 100
my_apps.filter['emax'] = 500000
my_apps.filter['zmax'] = 90
my_apps.filter['tmin'] = 239557417
my_apps.filter['tmax'] = 302572802
my_apps.filter['infile'] = '@./data/binned_events.txt'
my_apps.filter['outfile'] = './data/3C279_front_filtered.fits'

Once this is done, we can run **gtselect**:

In [ ]:
my_apps.filter.run()

Now, we select the back events and run it again:

In [ ]:
my_apps.filter['evtype'] = 2
my_apps.filter['outfile'] = './data/3C279_back_filtered.fits'

In [ ]:
my_apps.filter.run()

Now, we need to find the GTIs for each data set (front and back). This is accessed within python via the `maketime` object:

In [ ]:
# Front
my_apps.maketime['scfile'] = './data/L181126210218F4F0ED2738_SC00.fits'
my_apps.maketime['filter'] = '(DATA_QUAL>0)&&(LAT_CONFIG==1)'
my_apps.maketime['roicut'] = 'no'
my_apps.maketime['evfile'] = './data/3C279_front_filtered.fits'
my_apps.maketime['outfile'] = './data/3C279_front_filtered_gti.fits'

In [ ]:
my_apps.maketime.run()

Similar for the back:

In [ ]:
# Back
my_apps.maketime['evfile'] = './data/3C279_back_filtered.fits'
my_apps.maketime['outfile'] = './data/3C279_back_filtered_gti.fits'

In [ ]:
my_apps.maketime.run()

# Livetime and Counts Cubes

### Livetime Cube

We can now compute the livetime cube. We only need to do this once since in this case we made the exact same time cuts and used the same GTI filter on front and back datasets.

In [ ]:
my_apps.expCube['evfile'] = './data/3C279_front_filtered_gti.fits'
my_apps.expCube['scfile'] = './data/L181126210218F4F0ED2738_SC00.fits'
my_apps.expCube['outfile'] = './data/3C279_front_ltcube.fits'
my_apps.expCube['zmax'] = 90
my_apps.expCube['dcostheta'] = 0.025
my_apps.expCube['binsz'] = 1

In [ ]:
my_apps.expCube.run()

### Counts Cube

The counts cube is the counts from our data file binned in space and energy. All of the steps above use a circular ROI (or a cone, really).

Once you switch to binned analysis, you start doing things in squares. Your counts cube can only be as big as the biggest square that can fit in the circular ROI you already selected.

We start with front events:

In [ ]:
my_apps.evtbin['evfile'] = './data/3C279_front_filtered_gti.fits'
my_apps.evtbin['outfile'] = './data/3C279_front_ccube.fits'
my_apps.evtbin['algorithm'] = 'CCUBE'
my_apps.evtbin['nxpix'] = 100
my_apps.evtbin['nypix'] = 100
my_apps.evtbin['binsz'] = 0.2
my_apps.evtbin['coordsys'] = 'CEL'
my_apps.evtbin['xref'] = 193.98
my_apps.evtbin['yref'] = -5.82
my_apps.evtbin['axisrot'] = 0
my_apps.evtbin['proj'] = 'AIT'
my_apps.evtbin['ebinalg'] = 'LOG'
my_apps.evtbin['emin'] = 100
my_apps.evtbin['emax'] = 500000
my_apps.evtbin['enumbins'] = 37

In [ ]:
my_apps.evtbin.run()

And then for the back events:

In [ ]:
my_apps.evtbin['evfile'] = './data/3C279_back_filtered_gti.fits'
my_apps.evtbin['outfile'] = './data/3C279_back_ccube.fits'

In [ ]:
my_apps.evtbin.run()

# Exposure Maps

The binned exposure map is an exposure map binned in space and energy.

We first need to import the python version of `gtexpcube2`, which doesn't have a gtapp version by default. This is easy to do (you can import any of the command line tools into python this way). Then, you can check out the parameters with the `pars` function.

In [ ]:
from GtApp import GtApp
expCube2= GtApp('gtexpcube2','Likelihood')
expCube2.pars()

Here, we generate exposure maps for the entire sky. 

In [ ]:
expCube2['infile'] = './data/3C279_front_ltcube.fits'
expCube2['cmap'] = 'none'
expCube2['outfile'] = './data/3C279_front_BinnedExpMap.fits'
expCube2['irfs'] = 'P8R3_SOURCE_V2'
expCube2['evtype'] = '1'
expCube2['nxpix'] = 1800
expCube2['nypix'] = 900
expCube2['binsz'] = 0.2
expCube2['coordsys'] = 'CEL'
expCube2['xref'] = 193.98
expCube2['yref'] = -5.82
expCube2['axisrot'] = 0
expCube2['proj'] = 'AIT'
expCube2['ebinalg'] = 'LOG'
expCube2['emin'] = 100
expCube2['emax'] = 500000
expCube2['enumbins'] = 37

In [ ]:
expCube2.run()

In [ ]:
expCube2['infile'] = './data/3C279_front_ltcube.fits'
expCube2['outfile'] = './data/3C279_back_BinnedExpMap.fits'
expCube2['evtype'] = '2'

In [ ]:
expCube2.run()

# Compute Source Maps

The source maps step convolves the LAT response with your source model, generating maps for each source in the model for use in the likelihood calculation.

We use the same [XML](https://fermi.gsfc.nasa.gov/ssc/data/analysis/scitools/data/BinnedLikelihood/3C279_input_model.xml) file as in the standard [binned likelihood](https://fermi.gsfc.nasa.gov/ssc/data/analysis/scitools/binned_likelihood_tutorial.html) analysis.

You should also download the recommended models for a normal point source analysis `gll_iem_v07.fits` and `iso_P8R3_SOURCE_V2_v1.txt`.

These three files can be downloaded by running the code cell below:

In [ ]:
!wget https://fermi.gsfc.nasa.gov/ssc/data/analysis/software/aux/4fgl/gll_iem_v07.fits
!wget https://fermi.gsfc.nasa.gov/ssc/data/analysis/software/aux/4fgl/iso_P8R3_SOURCE_V2_v1.txt
!wget https://fermi.gsfc.nasa.gov/ssc/data/analysis/scitools/data/BinnedLikelihood/3C279_input_model.xml

In [ ]:
!mv *.xml ./data

Note that the files `gll_iem_v07.fits` and `iso_P8R3_SOURCE_V2_v1.txt` must be in your current working directory for the next steps to work.

We compute the front events:

In [ ]:
my_apps.srcMaps['expcube'] = './data/3C279_front_ltcube.fits'
my_apps.srcMaps['cmap'] = './data/3C279_front_ccube.fits'
my_apps.srcMaps['srcmdl'] = './data/3C279_input_model.xml'
my_apps.srcMaps['bexpmap'] = './data/3C279_front_BinnedExpMap.fits'
my_apps.srcMaps['outfile'] = './data/3C279_front_srcmap.fits'
my_apps.srcMaps['irfs'] = 'P8R3_SOURCE_V2'
my_apps.srcMaps['evtype'] = '1'

In [ ]:
my_apps.srcMaps.run()

And similarly, the back events:

In [ ]:
my_apps.srcMaps['expcube'] = './data/3C279_front_ltcube.fits'
my_apps.srcMaps['cmap'] = './data/3C279_back_ccube.fits'
my_apps.srcMaps['srcmdl'] = './data/3C279_input_model.xml'
my_apps.srcMaps['bexpmap'] = './data/3C279_back_BinnedExpMap.fits'
my_apps.srcMaps['outfile'] = './data/3C279_back_srcmap.fits'
my_apps.srcMaps['irfs'] = 'P8R3_SOURCE_V2'
my_apps.srcMaps['evtype'] = '2'

In [ ]:
my_apps.srcMaps.run()

# Run the Likelihood Analysis

First, import the BinnedAnalysis and SummedAnalysis libraries. Then, create a likelihood object for both the front and the back datasets. For more details on the pyLikelihood module, check out the [pyLikelihood Usage Notes](https://fermi.gsfc.nasa.gov/ssc/data/analysis/scitools/python_usage_notes.html).

In [ ]:
import pyLikelihood
from BinnedAnalysis import *
from SummedLikelihood import *

front = BinnedObs(srcMaps='./data/3C279_front_srcmap.fits',binnedExpMap='./data/3C279_front_BinnedExpMap.fits',expCube='./data/3C279_front_ltcube.fits',irfs='CALDB')
likefront = BinnedAnalysis(front,'./data/3C279_input_model.xml',optimizer='NewMinuit')
back = BinnedObs(srcMaps='./data/3C279_back_srcmap.fits',binnedExpMap='./data/3C279_back_BinnedExpMap.fits',expCube='./data/3C279_front_ltcube.fits',irfs='CALDB')
likeback = BinnedAnalysis(back,'./data/3C279_input_model.xml',optimizer='NewMinuit')

Then, create the summedlikelihood object and add the two likelihood objects, one for the front selection and the second for the back selection.

In [ ]:
summed_like = SummedLikelihood()
summed_like.addComponent(likefront)
summed_like.addComponent(likeback)

Perform the fit and print out the results:

In [ ]:
summedobj = pyLike.NewMinuit(summed_like.logLike)
summed_like.fit(verbosity=0,covar=True,optObject=summedobj)

Print TS for 3C 279 (4FGL J1256.1-0547):

In [ ]:
summed_like.Ts('4FGL J1256.1-0547')

We can now compare to the standard [binned likelihood](https://fermi.gsfc.nasa.gov/ssc/data/analysis/scitools/binned_likelihood_tutorial.html) analysis that uses only one data set containing both Front and Back event types that are represented by a single, combined IRF set. You will need to download the files created in that analysis thread or rerun this python tutorial with the combined dataset `(evtype=3)`.

For your convenience, the files can be obtained from the code cell below:

In [ ]:
!wget https://fermi.gsfc.nasa.gov/ssc/data/analysis/scitools/data/BinnedLikelihood/3C279_binned_srcmaps.fits
!wget https://fermi.gsfc.nasa.gov/ssc/data/analysis/scitools/data/BinnedLikelihood/3C279_binned_allsky_expcube.fits
!wget https://fermi.gsfc.nasa.gov/ssc/data/analysis/scitools/data/BinnedLikelihood/3C279_binned_ltcube.fits

In [ ]:
!mv *.fits ./data

In [ ]:
all = BinnedObs(srcMaps='./data/3C279_binned_srcmaps.fits',binnedExpMap='./data/3C279_binned_allsky_expcube.fits',expCube='./data/3C279_binned_ltcube.fits',irfs='CALDB')
likeall = BinnedAnalysis(all,'./data/3C279_input_model.xml',optimizer='NewMinuit')

Perform the fit and print out the results:

In [ ]:
likeallobj = pyLike.NewMinuit(likeall.logLike)
likeall.fit(verbosity=0,covar=True,optObject=likeallobj)

Print TS for 3C 279 (4FGL J1256.1-0547):

In [ ]:
likeall.Ts('4FGL J1256.1-0547')

The TS for the front + back analysis is 29261.558, a bit lower than what we found for the separate front and back analysis 30191.550.

The important difference is that in the separated version of the analysis each event type has a dedicated response function set instead of using the averaged Front+Back response. This should increase the sensitivity, and therefore, the TS value.